In [78]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import sympy as sp
from IPython.display import display

# Model Project - The Small Open Economy

Set-up is as follows: GDP, the national identity, is defined as the sum of consumption, investment and net-exports:
$$Y = C + I + NX$$   
GNI adds to this rental income from foreign capital:
$$Y + rF =  C + I + NX + rF$$    
Savings are Gross National Income minus consumption: 
$$S_t = Y_t + rF_t - C_t$$
Savings help accumulate capital - either domestically, or abroad: 
$$S_t = I_t + F_{t+1} - F_t$$
Capital evolves as per usual (at first simplified by assuming zero depreciation ... assumption later relaxed):
$$K_{t+1} = I_t + K_t$$
Combining terms we see that:
$$K_{t+1} = S_t - F_{t+1} + F_t + K_t ↔ K_{t+1} + F_{t+1} = S_t + F_t + K_t$$
Wealth can be domestically owned, or foreign:
$$V_t = K_t + F_t$$
This means that wealth tomorrow is:
$$V_{t+1} = V_t + S_t$$
$S_t = s (Y_t + rF_t)$, where saving, s, is a fraction of income set between 0 and 1.  Our production function is initially defined as CD (we'll show other variants later) $Y_t = AK_t^{\alpha} L_t^{1-\alpha}$. Markets are competitive. $r=r^w$ at all times. This implies a constant level of capital (barring changes to $\alpha$ or A). 
$MP_K$ = $f'k$ = rental rate of capital. Thus, $f'k = \alpha A K_t^{\alpha -1} L_t^{1-\alpha}$, $r^w = \alpha A k_t^{\alpha - 1}$, and $\bar{k} = \left(\frac{r^w}{\alpha A}\right)^{\frac{1}{\alpha-1}}$. Wages are constant by the same reasoning. $MP_L$ = $f'l$ = $\bar{w}$ = $(1-{\alpha})$ $\frac{Y}{L}$ = $(1-{\alpha})A \bar{k}^{\alpha}$. Total returns to scale tell us that $Y_t = rK_t + wL_t$. 

We use sympy to derive the law of motion for wealth and then proceed to the numerical analysis the assignment asks for. 

Equations come from: https://web.econ.ku.dk/dalgaard/makro2_2008/Slides/lecture4_cm.pdf

In [80]:
# Defining the necessary symbols
Y_t, C_t, I_t, r, rw, F_t, K_t, S, s, V, V1, L_t, w, n, L_t1, v_t = sp.symbols(
    'Y_t C_t I_t r r^w F_t K_t S s V_t V_{t+1} L_t w n L_{t+1} v_t'
)

# Equation setup
Y = w * L_t + r * K_t         # Production function
GNI = Y + r * F_t               # Gross National Income
S = s * GNI                     # Savings
V = K_t + F_t                   # Wealth at time t
V1 = S + V                      # Wealth at time t+1

# Expand all expressions 
V1_expanded = V1.expand()

# Substitute V_t for K_t + F_t in the expanded V1
V1_substituted = V1_expanded.subs({K_t + F_t: V_t})

# Collect terms to consolidate expressions around V_t
V1_collected = sp.collect(V1_substituted, V_t)

# Explicitly factor out (1 + r * s) from the terms involving V_t
V1_intermed = V1_collected.subs(r * s * K_t + r * s * F_t, (r * s + 1) * V_t).simplify()

# Define L_t+1 
L_1 = L_t * (1 + n)

# Define per capita wealth today and tomorrow 
v_t = V_t / L_t
v_1 = V1_intermed / L_1

# This gives us the Law of Motion for Wealth (LoM)
LoM = v_1.subs(r * s * K_t + r * s * F_t, (r * s + 1) * V_t).simplify()

LoM

(L_t*s*w + V_t*(r*s + 1) + V_t)/(L_t*(n + 1))

We divide this thru by $L_t$ to arrive at: $v_{t+1} = \frac{sw}{1+n}+\frac{1+sr}{1+n}v_t$

In the Steady State, $v_{t+1}=v_t=v^*$

In [82]:
s, w, r, n, v = sp.symbols('s w r n v')
SteadyState = sp.Eq((s * w) / (1 + n) + ((1 + s * r) / (1 + n)) * v, v)
vstar = sp.solve(SteadyState, v)
vstar

[s*w/(n - r*s)]

## Numerical Analysis 

### Ideas:
Set up model as a class 
- show effects of different production functions (Cobb Douglas, CES .. )
- show effects of Technology shock
- plot response curve to shocks / changes in saving etc.

- Simulate to get an idea of length of transition period to Steady State
- Draw phase diagrame 